In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [2]:
import pickle
import econml
import itertools
from collections import Counter
from tqdm import tqdm

from rashomon import hasse
from rashomon import loss
from rashomon import counter
from rashomon import metrics
from rashomon import extract_pools
from rashomon.aggregate import (RAggregate_profile, RAggregate,
    find_profile_lower_bound, find_feasible_combinations, remove_unused_poolings, subset_data)
from rashomon.sets import RashomonSet, RashomonProblemCache, RashomonSubproblemCache
from rashomon.aggregate import find_te_het_partitions

from econml.grf import CausalForest
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn import linear_model

from copy import deepcopy

%load_ext autoreload
%autoreload 2

In [3]:
def generate_data(mu, var, n_per_pol, all_policies, pi_policies, M):
    num_data = num_policies * n_per_pol
    X = np.zeros(shape=(num_data, M))
    D = np.zeros(shape=(num_data, 1), dtype='int_')
    y = np.zeros(shape=(num_data, 1))  + np.inf
    mu_true = np.zeros(shape=(num_data, 1))

    idx_ctr = 0
    for k, profile in enumerate(profiles):
        policies_k = policies_profiles[k]

        for idx, policy in enumerate(policies_k):
            policy_idx = [i for i, x in enumerate(all_policies) if x == policy]
            
            if pi_policies[k] is None and np.isnan(mu[k]):
                continue
                
            pool_id = pi_policies[k][idx]
            mu_i = mu[k][pool_id]
            var_i = var[k][pool_id]
            y_i = np.random.normal(mu_i, var_i, size=(n_per_pol, 1))

            start_idx = idx_ctr * n_per_pol
            end_idx = (idx_ctr + 1) * n_per_pol

            X[start_idx:end_idx, ] = policy
            D[start_idx:end_idx, ] = policy_idx[0]
            y[start_idx:end_idx, ] = y_i
            mu_true[start_idx:end_idx, ] = mu_i

            idx_ctr += 1

    absent_idx = np.where(np.isinf(y))[0]
    X = np.delete(X, absent_idx, 0)
    y = np.delete(y, absent_idx, 0)
    D = np.delete(D, absent_idx, 0)
    mu_true = np.delete(mu_true, absent_idx, 0)

    return X, D, y, mu_true

# Setup

## Parameters

We set up all the parameters of the simulation as well as the encodings of profiles, policies, treatment, etc. below.

In [4]:
M = 4
R = np.array([2, 4, 5, 5])


# (0, 1, 1, 1)
sigma_0 = np.array([[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]
                   ])
mu_0 = np.array([0])
var_0 = np.array([0])

# (1, 1, 1, 1)
sigma_1 = np.array([[np.inf, np.inf, np.inf],
                    [0, 0, np.inf],
                    [1, 0, 1],
                    [1, 1, 0],
                   ])
mu_1 = np.array([2, 4, 2, 0, 3, 5, 7, 1, 1, -1, -1, -2])
var_1 = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


interested_profiles = [(0, 1, 1, 1), (1, 1, 1, 1)]

sigma_tmp = [sigma_0, sigma_1]
mu_tmp = [mu_0, mu_1]
var_tmp = [var_0, var_1]

\begin{align*}
    \text{Control} & \\
    (0,\ 1:3,\  1:4,\  1:4) &= 0 \\
    \text{Treatment} & \\
    (1,\  1,\  1:2,\  1:3) &= 2 \\
    (1,\  1,\  1:2,\  4) &= 4 \\
    (1,\  1,\  3:4,\  1:3) &= 2 \\
    (1,\  1,\  3:4,\  4) &= 0 \\
    (1,\  2,\  1:2,\  1:3) &= 3 \\
    (1,\  2,\  1:2,\  4) &= 5 \\
    (1,\  2,\  3:4,\  1:3) &= 7 \\
    (1,\  2,\  3:4,\  4) &= 1 \\
    (1,\  3,\  1:2,\  1:3) &= 1 \\
    (1,\  3,\  1:2,\  4) &= -1 \\
    (1,\  3,\  3:4,\  1:3) &= -1 \\
    (1,\  3,\  3:4,\  4) &= -2
\end{align*}

In [5]:
num_profiles = 2**M
profiles, profile_map = hasse.enumerate_profiles(M)
all_policies = hasse.enumerate_policies(M, R)
num_policies = len(all_policies)

interested_profile_idx = []
sigma = []
mu = []
var = []
for k, profile in enumerate(profiles):
    sigma_k = None
    mu_k = np.nan
    var_k =  np.nan
    for i, p in enumerate(interested_profiles):
        if p == profile:
            sigma_k = sigma_tmp[i]
            mu_k = mu_tmp[i]
            var_k = var_tmp[i]
            break
    sigma.append(sigma_k)
    mu.append(mu_k)
    var.append(var_k)

In [6]:
# Identify the pools
policies_profiles = {}
policies_profiles_masked = {}
policies_ids_profiles = {}
pi_policies = {}
pi_pools = {}
for k, profile in enumerate(profiles):

    policies_temp = [(i, x) for i, x in enumerate(all_policies) if hasse.policy_to_profile(x) == profile]
    unzipped_temp = list(zip(*policies_temp))
    policies_ids_k = list(unzipped_temp[0])
    policies_k = list(unzipped_temp[1])
    policies_profiles[k] = deepcopy(policies_k)
    policies_ids_profiles[k] = policies_ids_k

    profile_mask = list(map(bool, profile))

    # Mask the empty arms
    for idx, pol in enumerate(policies_k):
        policies_k[idx] = tuple([pol[i] for i in range(M) if profile_mask[i]])
    policies_profiles_masked[k] = policies_k


    # profile_idx = None
    # for idx, p in enumerate(interested_profiles):
    #     if p == profile:
    #         profile_idx = idx
    # if profile_idx is None:
    #     pi_policies[k] = None
    #     pi_pools[k] = None
    #     continue
    if sigma[k] is None:
        pi_policies[k] = None
        pi_pools[k] = None
        continue

    if np.sum(profile) > 0:
        pi_pools_k, pi_policies_k = extract_pools.extract_pools(policies_k, sigma[k])
        if len(pi_pools_k.keys()) != mu[k].shape[0]:
            print(pi_pools_k)
            print(f"Profile {k}. Expected {len(pi_pools_k.keys())} pools. Received {mu[k].shape[0]} means.")
        pi_policies[k] = pi_policies_k
        # pi_pools_k has indicies that match with policies_profiles[k]
        # Need to map those indices back to all_policies
        pi_pools[k] = {}
        for x, y in pi_pools_k.items():
            y_full = [policies_profiles[k][i] for i in y]
            y_agg = [all_policies.index(i) for i in y_full]
            pi_pools[k][x] = y_agg
    else:
        pi_policies[k] = {0: 0}
        pi_pools[k] = {0: [0]}

best_per_profile = [np.max(mu_k) for mu_k in mu]
true_best_profile = np.nanargmax(best_per_profile)
true_best_profile_idx = int(true_best_profile)
true_best_effect = np.max(mu[true_best_profile])
true_best = pi_pools[true_best_profile][np.argmax(mu[true_best_profile])]
min_dosage_best_policy = metrics.find_min_dosage(true_best, all_policies)

# The transformation matrix for Lasso
G = hasse.alpha_matrix(all_policies)

In [7]:
trt_arm_idx = 0

ctl_profile_idx = 7
trt_profile_idx = 15

ctl_profile = (0, 1, 1, 1)
trt_profile = (1, 1, 1, 1)

# Subset data for interested profiles
trt_policies_ids = policies_ids_profiles[trt_profile_idx] # These are the policy ids (15 corresponds to 1, 1, 1, 1)
ctl_policies_ids = policies_ids_profiles[ctl_profile_idx]
tc_policies_ids = trt_policies_ids + ctl_policies_ids

trt_policies = policies_profiles_masked[trt_profile_idx]
ctl_policies = policies_profiles_masked[ctl_profile_idx]

trt_pools, trt_pools_policies = extract_pools.extract_pools(trt_policies, sigma[trt_profile_idx])
ctl_pools, ctl_pools_policies = extract_pools.extract_pools(ctl_policies, sigma[ctl_profile_idx])

D_trt = np.array(list(trt_pools_policies.keys()))
D_ctl = np.array(list(ctl_pools_policies.keys()))

D_trt_pooled = [trt_pools_policies[pol_id] for pol_id in D_trt]
D_ctl_pooled = [ctl_pools_policies[pol_id] for pol_id in D_ctl]
y_trt = mu[trt_profile_idx][D_trt_pooled]
y_ctl = mu[ctl_profile_idx][D_ctl_pooled]

X_trt = np.array(policies_profiles[trt_profile_idx])[:, 1:]

te_true = y_trt - y_ctl
max_te = np.max(te_true)
max_te_policies_p = D_trt[np.where(te_true == max_te)]
max_te_policies = [policies_ids_profiles[trt_profile_idx][x] for x in max_te_policies_p]
min_dosage_best_te = metrics.find_min_dosage(max_te_policies, all_policies)

## Generate data

In [8]:
np.random.seed(3)

n_per_pol = 10

# Generate data
X, D, y, mu_true = generate_data(mu, var, n_per_pol, all_policies, pi_policies, M)
policy_means = loss.compute_policy_means(D, y, num_policies)
# The dummy matrix for Lasso
D_matrix = hasse.get_dummy_matrix(D, G, num_policies)

trt_arm_idx = 0
feature_idx = list(np.arange(0, trt_arm_idx)) + list(np.arange(trt_arm_idx+1, M))

T = np.zeros(shape=y.shape)
T[X[:, trt_arm_idx] > 0] = 1

y_0d = y.reshape((-1,))
X_cf = X[:, feature_idx]

X_trt_subset = X[X[:, trt_arm_idx] > 0, :]
X_trt_subset = X_trt_subset[:, feature_idx]
y_trt_subset = y[X[:, trt_arm_idx] > 0]


D_trt_subset = D[X[:, trt_arm_idx] > 0]
D_matrix_trt_subset = D_matrix[X[:, trt_arm_idx] > 0, :]
D_matrix_ctl_subset = D_matrix[X[:, trt_arm_idx] == 0, :]

D_trt_univ = np.array([policies_ids_profiles[trt_profile_idx][x] for x in D_trt]).reshape((-1, 1))
D_ctl_univ = np.array([policies_ids_profiles[ctl_profile_idx][x] for x in D_ctl]).reshape((-1, 1))
D_matrix_trt = hasse.get_dummy_matrix(D_trt_univ, G, num_policies)
D_matrix_ctl = hasse.get_dummy_matrix(D_ctl_univ, G, num_policies)


policy_means = loss.compute_policy_means(D, y, num_policies)

mask = np.isin(D, tc_policies_ids)
D_tc = D[mask].reshape((-1,1))
y_tc = y[mask].reshape((-1,1))

# Estimation

## Causal Forests

In this section, we fit a Causal Forest Estimator and extract the partitions from each individual tree in the estimator.

In [9]:
# https://econml.azurewebsites.net/_autosummary/econml.grf.CausalForest.html?highlight=causalforest#econml.grf.CausalForest
est = CausalForest(criterion="het", n_estimators=500,
                   min_samples_leaf=10,
                   # max_depth=None,
                   min_samples_split=10,
                   random_state=3,
                  )

# est.fit(X_cf, X[:, trt_idx], y_0d)
est.fit(X_cf, T, y_0d)
# est.fit(X, 1 + np.zeros(T.shape), y_0d)

CausalForest(criterion='het', min_samples_leaf=10, n_estimators=500,
             random_state=3)

In [10]:
# Extract the trees from the CRF
trees = est.estimators_

# This function takes a tree and creates a dictionary based representation where each non-leaf node is a dictionary with dictionary children, and leaves are sets of data points.
# The decision_path() call in line ___ returns a sparse matrix that encodes the decision path for each of the data points in X_cf, i.e. it lets us
# reconstruct the partition by following how each profile is categorized. The code in lines iteratively fills in the dictionary that represents the 
# binary tree by running through the path (each entry in cx.row) for each 
def create_dict_rep(tree):
    root = {}
    head = root

    # Takes the sparse matrix from the decision path of each data point and then turns it into a list of coordinates.
    cx = tree.tree_.decision_path(X_cf).tocoo()

    # We iterate through the list of coordinates and construct the tree by traversing until we hit a leaf node and then adding the data point to the set in that leaf.
    for i in range(len(cx.row)):
        if cx.col[i] not in head:
            head[cx.col[i]] = {}
        head = head[cx.col[i]]
        if (i != len(cx.row) - 1 and cx.row[i] != cx.row[i + 1]) or (i == len(cx.row) - 1):
            if "*" not in head:
                head["*"] = set()
            # The "*" indicates that we have gotten to the end of a decision path, so we add the data point into the set at this leaf.
            head["*"].add(tuple(X_cf[cx.row[i - 1], :].astype(int)))
            head = root
    return root

# This function recursively traverses through the tree and constructs a list of the pools corresonding to the leaves.
def traverse(tree):
    to_return = []
    for key in tree.keys():
        if key == "*":
            return [tree[key]]
        else:
            to_return += traverse(tree[key])
    return to_return

# Create for each tree in the CRF trees.
crf_trees = []
for i in range(len(trees)):
    crf_trees.append(traverse(create_dict_rep(trees[i])))

The following chunk of code creates a Counter to keep track of what pools are in each tree partition for comparison later with the RPS partitions. (I am aware that this implementation is inefficient, but for the purposes of this simulation it isn't too important.)

In [11]:
all_pools_crf_counters = {}

# Loop through all the trees and create a Counter for each tree.
for tree_idx in range(len(crf_trees)):
    c = Counter()
    for value in crf_trees[tree_idx]:
        to_add = tuple(sorted(value))
        c.update([to_add])
    all_pools_crf_counters[tree_idx] = c

In [12]:
te_cf = est.predict(X_trt)

cf_res = metrics.compute_te_het_metrics(
                            te_true, te_cf,
                            max_te, max_te_policies,
                            D_trt, policies_ids_profiles[trt_profile_idx]
                        )

cf_res

{'mse_te': 1.2983811452086564,
 'max_te_est': 6.724452263969456,
 'max_te_err': 0.2755477360305436,
 'iou': 0.14285714285714285,
 'conf_matrix': array([[0.8, 0. , 0.2],
        [0. , 0. , 1. ],
        [0. , 0. , 1. ]])}

## Rashomon Sets

Use the following chunk of code below to tune the size of the Rashomon Set.

In [13]:
H = np.inf

theta = 2.5
reg = 1e-1
R_set, rashomon_profiles = RAggregate(M, R, H, D, y, theta, reg,
                                     verbose=True, bruteforce=True
                                     )

print(len(R_set))

Skipping profile (0, 0, 0, 0)
Skipping profile (0, 0, 0, 1)
Skipping profile (0, 0, 1, 0)
Skipping profile (0, 0, 1, 1)
Skipping profile (0, 1, 0, 0)
Skipping profile (0, 1, 0, 1)
Skipping profile (0, 1, 1, 0)
(0, 1, 1, 1) 2.0402404523030584
Brute forcing
Took 0.2992069721221924 s when brute forcing
220
Skipping profile (1, 0, 0, 0)
Skipping profile (1, 0, 0, 1)
Skipping profile (1, 0, 1, 0)
Skipping profile (1, 0, 1, 1)
Skipping profile (1, 1, 0, 0)
Skipping profile (1, 1, 0, 1)
Skipping profile (1, 1, 1, 0)
(1, 1, 1, 1) 2.5
Brute forcing
Took 0.2734489440917969 s when brute forcing
36
Finding feasible combinations
Min = 1.7865754084828451, Max = 4.282754080538238
520


In [14]:
model_losses = []
model_sizes = []

for r_set in R_set:
    pi_policies_profiles_r = {}

    loss_r = 0
    size_r = 0
    
    for profile, model_prof in enumerate(r_set):
        sigma_r_prof = rashomon_profiles[profile].sigma[model_prof]
        loss_r_prof = rashomon_profiles[profile].loss[model_prof]
        size_r_prof = rashomon_profiles[profile].pools[model_prof]

        loss_r += loss_r_prof

        if sigma_r_prof is None and loss_r_prof == 0:
            pass
        else:
            size_r += size_r_prof
    
    model_losses.append(loss_r)
    model_sizes.append(size_r)

model_losses = np.array(model_losses)
model_sizes = np.array(model_sizes)

q_0 = np.min(model_losses)

print(f"Best model loss {q_0}")

Best model loss 1.7865754084828451


## Intersection of Treatment and Control Partitions with CRF Partitions

The function below takes the Rashomon Set and extracts the poolings and transforms them to the form of the counters in all_pools_crf_counters so they can be compared. We then compute the intersection.

In [15]:
def create_rset_poolings(theta, reg, M=M, R=R, H=H, D=D, y=y, verbose=True):
    H = np.inf
    print(f"Running RAggregate with theta = {theta} and reg = {reg}")
    R_set, rashomon_profiles = RAggregate(M, R, H, D, y, theta, reg,
                                        verbose=verbose,
                                        )

    print("Length of the RPS is", len(R_set))

    # Create the Poolings for the Treatment and Control Profiles
    all_r_pools_trt = []
    all_r_pools_ctl = []

    # For each rashomon set, extract the pools for the treatment and control separately and append them to the all_r_pools_trt, and all_r_pools_ctl.
    for idx, r_set in enumerate(R_set):

        sigma_trt_R_set_idx = r_set[trt_profile_idx]
        sigma_trt_i = rashomon_profiles[trt_profile_idx].sigma[sigma_trt_R_set_idx]
        sigma_ctl_R_set_idx = r_set[ctl_profile_idx]
        sigma_ctl_i = rashomon_profiles[ctl_profile_idx].sigma[sigma_ctl_R_set_idx]

        trt_pools_0, _ = extract_pools.extract_pools(trt_policies, sigma_trt_i)
        ctl_pools_0, _ = extract_pools.extract_pools(ctl_policies, sigma_ctl_i) # This gives dictionary of pools --> this is the partition
        
        trt_pools = hasse.profile_ids_to_univ_ids(trt_pools_0, trt_policies_ids)
        ctl_pools = hasse.profile_ids_to_univ_ids(ctl_pools_0, ctl_policies_ids)

        all_r_pools_trt.append(trt_pools)
        all_r_pools_ctl.append(ctl_pools)

    # Create dictionaries that map between ids and policies for both treatment and control
    id_to_policy_dict_trt = {i: x for i, x in enumerate(all_policies) if hasse.policy_to_profile(x) == (1, 1, 1, 1)}
    id_to_policy_dict_ctl = {i: x for i, x in enumerate(all_policies) if hasse.policy_to_profile(x) == (0, 1, 1, 1)}
    all_r_pools_trt_counters, all_r_pools_ctl_counters = {}, {}

    # For each rashomon set, we make a Counter for both the treatment and control poolings and insert in a dictionary.
    for r_set_idx in range(len(all_r_pools_trt)):
        c_trt = Counter()
        c_ctl = Counter()
        for key, value in all_r_pools_trt[r_set_idx].items():
            to_add = list(map(tuple, [id_to_policy_dict_trt[idx][1:] for idx in value]))
            to_add = tuple((sorted(to_add)))
            all_r_pools_trt[r_set_idx][key] = to_add
            c_trt.update([to_add])

        for key, value in all_r_pools_ctl[r_set_idx].items():
            to_add = tuple(map(tuple, [id_to_policy_dict_ctl[idx][1:] for idx in value]))
            to_add = tuple(sorted(to_add))
            all_r_pools_ctl[r_set_idx][key] = to_add
            c_ctl.update([to_add])

        all_r_pools_trt_counters[r_set_idx] = c_trt
        all_r_pools_ctl_counters[r_set_idx] = c_ctl
    
    return all_r_pools_trt_counters, all_r_pools_ctl_counters

In [16]:
# Compute the intersection between the partitions from the CRF trees and the partitions from the Rashomon Set.
# This can take a long time depending on the size of your RPS. It takes 5-10 minutes for 100 trees and 60000+ models in the Rashomon Set.

all_r_pools_trt_counters, all_r_pools_ctl_counters = create_rset_poolings(3, 0.1)
intersection_trt, intersection_ctl = [], []

for key, pool in tqdm(all_pools_crf_counters.items()):
    if pool in all_r_pools_trt_counters.values():
        intersection_trt.append(pool)
    if pool in all_r_pools_ctl_counters.values():
        intersection_ctl.append(pool)

print("Intersections from Treatment Partitions:", len(intersection_trt), "\nIntersections from Control Partitions:", len(intersection_ctl))

Running RAggregate with theta = 3 and reg = 0.1
Skipping profile (0, 0, 0, 0)
Skipping profile (0, 0, 0, 1)
Skipping profile (0, 0, 1, 0)
Skipping profile (0, 0, 1, 1)
Skipping profile (0, 1, 0, 0)
Skipping profile (0, 1, 0, 1)
Skipping profile (0, 1, 1, 0)
(0, 1, 1, 1) 2.5402404523030584
Adaptive
Took 1.2091519832611084 s adaptively
238
Skipping profile (1, 0, 0, 0)
Skipping profile (1, 0, 0, 1)
Skipping profile (1, 0, 1, 0)
Skipping profile (1, 0, 1, 1)
Skipping profile (1, 1, 0, 0)
Skipping profile (1, 1, 0, 1)
Skipping profile (1, 1, 1, 0)
(1, 1, 1, 1) 3.0
Adaptive
Took 1.278930902481079 s adaptively
92
Finding feasible combinations
Min = 1.7865754084828451, Max = 5.983622437621808
Length of the RPS is 3076


100%|█████████████████████████████████████████| 500/500 [00:06<00:00, 79.79it/s]

Intersections from Treatment Partitions: 8 
Intersections from Control Partitions: 8


## Permissibility of CRF Partitions

The following code checks to see if the trees in the Causal Random Forest admit permissible partitions. The $\texttt{is\_permissible()}$ function takes a partition as well as the array $R$ which parameterizes the arms of the problem. It outputs a truth value.

In [18]:
# R = np.array([2, 4, 5, 5])
from itertools import product

crf_R = R[1:]

# Computes if there is exactly one True among a boolean array.
def single_true(iterable):
    i = iter(iterable)
    return any(i) and not any(i)

# Checks to see if a pool satisfies the strongly convex criterion (is shaped like a box)
def not_strongly_convex(pool):
    min_array, max_array = [min(profile) for profile in zip(*pool)], [max(profile) for profile in zip(*pool)]
    arrays = [list(range(min_array[i], max_array[i] + 1)) for i in range(len(min_array))]
    for profile in product(*arrays):
        if profile not in pool:
            return True
        
    return False

# Checks if a partition, implemented as a set of pools, is permissible.
def is_permissible(partition, R):
    pools_look_up = dict()
    
    # Create pools_look_up, a dictionary that maps each profile to its pool.
    for pool in partition:
        if not_strongly_convex(pool):
            return False
        for item in pool:
            pools_look_up[item] = pool
    
    # For every profile and every profile in the same pool that differs by only one arm,
    # we loop through all the other pools and check to see if 
    for profile, pool in pools_look_up.items():
        # print("Current Profile:", profile)
        arrays = [list(range(1, element)) for element in R]
        for other_profile in pool:
            # print("---Other Profile:", other_profile)
            relative_diff = np.array(other_profile) - np.array(profile)

            # We want to skip profile comparisons that are not just a change in one arm.
            if not single_true(relative_diff != 0):
                # print("Skipping This Profile")
                continue
            nonzero_diff_indices = np.nonzero(relative_diff)[0]
            for nonzero_diff_idx in nonzero_diff_indices:
                arrays[nonzero_diff_idx] = [profile[nonzero_diff_idx]]
            for profile2 in product(*arrays):
                # print("------Search Profile 2:", profile2)
                new_profile = np.array(profile2) + relative_diff

                # If this profile does not exist, then we should continue
                if any(new_profile + 1 > R) or any(new_profile - 1 < 0):
                    continue

                # If this profile does exist but is not in the same pool as the profile it is being compared to, then we return False
                if profile2 not in pools_look_up[tuple(new_profile)]:
                    # print(f"Failed: {profile2} not in same pool as {tuple(new_profile)} even though {profile} in same pool as {other_profile}")
                    return False
    
    return True


In [19]:
# Running permissiblity on the list of CRF trees obtained from earlier in the notebook.
permissibility = [is_permissible(crf_trees[i], crf_R) for i in range(len(crf_trees))]
permissible_trees_idx = [i for i, e in enumerate(permissibility) if e != 0]

# This prints all of the permissible CRF tree partitions.
for idx in permissible_trees_idx:
    print(f"Tree {idx}")
    print(*crf_trees[idx], sep='\n')

Tree 3
{(1, 2, 1), (1, 2, 2), (1, 1, 1), (1, 1, 2)}
{(1, 1, 3), (1, 1, 4), (1, 2, 3), (1, 2, 4)}
{(1, 3, 1), (1, 3, 2), (1, 4, 1), (1, 4, 2)}
{(1, 4, 3), (1, 4, 4), (1, 3, 3), (1, 3, 4)}
{(2, 1, 1), (2, 1, 2), (2, 2, 1), (2, 2, 2)}
{(2, 2, 3), (2, 2, 4), (2, 1, 3), (2, 1, 4)}
{(2, 3, 1), (2, 3, 2), (2, 4, 1), (2, 4, 2)}
{(2, 4, 4), (2, 3, 3), (2, 3, 4), (2, 4, 3)}
{(3, 2, 1), (3, 2, 2), (3, 1, 1), (3, 1, 2)}
{(3, 1, 3), (3, 1, 4), (3, 2, 3), (3, 2, 4)}
{(3, 3, 1), (3, 3, 2), (3, 4, 1), (3, 4, 2)}
{(3, 4, 3), (3, 4, 4), (3, 3, 3), (3, 3, 4)}
Tree 10
{(1, 2, 1), (1, 2, 2), (1, 1, 1), (1, 1, 2)}
{(1, 1, 3), (1, 1, 4), (1, 2, 3), (1, 2, 4)}
{(1, 3, 1), (1, 3, 2), (1, 4, 1), (1, 4, 2)}
{(1, 4, 3), (1, 4, 4), (1, 3, 3), (1, 3, 4)}
{(2, 1, 1), (2, 1, 2), (2, 2, 1), (2, 2, 2)}
{(2, 2, 3), (2, 2, 4), (2, 1, 3), (2, 1, 4)}
{(2, 3, 1), (2, 3, 2), (2, 4, 1), (2, 4, 2)}
{(2, 4, 4), (2, 3, 3), (2, 3, 4), (2, 4, 3)}
{(3, 2, 1), (3, 2, 2), (3, 1, 1), (3, 1, 2)}
{(3, 1, 3), (3, 1, 4), (3, 2, 3), (3, 2,

## Producing Tables/Plots

The chunk of code below produces plots for this simulation. It performs 100 simulations and then . This assumes that you have run all the code blocks before and computed the Counter representations for the RPS (takes a long time so we reuse it here).

In [21]:
num_trees_to_try = np.array([20, 100, 500])
multipliers_to_try = np.array([1.1, 1.2, 1.3, 1.5])

# Finds suitable thetas to try based on the multipliers
def produce_thetas_to_try(multipliers, D_matrix, D, y):

    lasso_reg = 1e-1
    lasso = linear_model.Lasso(lasso_reg, fit_intercept=False)
    lasso.fit(D_matrix, y)
    alpha_est = lasso.coef_
    y_tva = lasso.predict(D_matrix)

    sqrd_err = mean_squared_error(y, y_tva)
    theta = sqrd_err + 1e-4 * np.linalg.norm(alpha_est, ord=1)

    R_set = []
    while len(R_set) == 0:
        reg = 1e-1
        R_set, rashomon_profiles = RAggregate(M, R, H, D, y, theta, reg,
                                                verbose=False, bruteforce=True
                                                )
        theta += 0.1

    model_losses = []
    model_sizes = []

    for r_set in R_set:

        loss_r = 0
        size_r = 0
        
        for profile, model_prof in enumerate(r_set):
            sigma_r_prof = rashomon_profiles[profile].sigma[model_prof]
            loss_r_prof = rashomon_profiles[profile].loss[model_prof]
            size_r_prof = rashomon_profiles[profile].pools[model_prof]

            loss_r += loss_r_prof

            if sigma_r_prof is None and loss_r_prof == 0:
                pass
            else:
                size_r += size_r_prof
        
        model_losses.append(loss_r)
        model_sizes.append(size_r)

    model_losses = np.array(model_losses)
    model_sizes = np.array(model_sizes)

    q_0 = np.min(model_losses)

    return q_0 * multipliers


# Runs the intersection and num_permissibile finding algorithms for all combinations of num_trees and thetas.
def produce_crf_counter(num_trees_to_try, thetas_to_try, D, y, X_cf, T, y_0d, parameters):
    trt_intersections, ctl_intersections, permissible_trees, size_of_r_sets = [], [], [], []
    for theta in thetas_to_try:
        # print(f"Running for theta = {theta}")
        all_r_pools_trt_counters, all_r_pools_ctl_counters = create_rset_poolings(theta, reg, D=D, y=y, verbose=False)
        for num_trees in num_trees_to_try:
            # print(f"Running for num_trees = {num_trees}")

            est = CausalForest(criterion="het", n_estimators=num_trees,
                            min_samples_leaf=10,
                            # max_depth=None,
                            min_samples_split=10,
                            random_state=3,
                            )
            
            est.fit(X_cf, T, y_0d)
            trees = est.estimators_

            crf_trees = []
            for i in range(len(trees)):
                crf_trees.append(traverse(create_dict_rep(trees[i])))

            all_pools_crf_counters = {}

            for tree_idx in range(len(crf_trees)):
                c = Counter()
                for value in crf_trees[tree_idx]:
                    to_add = tuple(sorted(value))
                    c.update([to_add])
                all_pools_crf_counters[tree_idx] = c

            intersection_trt, intersection_ctl = [], []

            for key, pool in tqdm(all_pools_crf_counters.items()):
                if pool in all_r_pools_trt_counters.values():
                    intersection_trt.append(pool)
                if pool in all_r_pools_ctl_counters.values():
                    intersection_ctl.append(pool)

            num_trt_intersections, num_ctl_intersections = len(intersection_trt), len(intersection_ctl)

            permissibility = [is_permissible(crf_trees[i], crf_R) for i in range(len(crf_trees))]
            permissible_trees_idx = [i for i, e in enumerate(permissibility) if e != 0]
            num_permissible_trees = len(permissible_trees_idx)

            parameters.append((theta, num_trees))
            trt_intersections.append(num_trt_intersections)
            ctl_intersections.append(num_ctl_intersections)
            permissible_trees.append(num_permissible_trees)
            size_of_r_sets.append(len(all_r_pools_trt_counters))

    size_of_r_sets = np.array(size_of_r_sets).reshape((len(thetas_to_try), len(num_trees_to_try)))
    trt_intersections = np.array(trt_intersections).reshape((len(thetas_to_try), len(num_trees_to_try)))
    ctl_intersections = np.array(ctl_intersections).reshape((len(thetas_to_try), len(num_trees_to_try)))
    permissible_trees = np.array(permissible_trees).reshape((len(thetas_to_try), len(num_trees_to_try)))

    return trt_intersections, ctl_intersections, permissible_trees, size_of_r_sets

# Runs sims for a given number of times
def run_sims(num_trees_to_try, multipliers_to_try, num_times=100):
    
    parameters = list(product(multipliers_to_try, num_trees_to_try))

    for i in range(1, num_times + 1):
        print(f"\n\nRunning Iteration {i}")
        np.random.seed(i)

        n_per_pol = 10

        # Generate data
        X, D, y, mu_true = generate_data(mu, var, n_per_pol, all_policies, pi_policies, M)

        D_matrix = hasse.get_dummy_matrix(D, G, num_policies)
        y_0d = y.reshape((-1,))
        X_cf = X[:, feature_idx]
        T = np.zeros(shape=y.shape)
        T[X[:, trt_arm_idx] > 0] = 1

        # Generate Thetas to Use
        thetas_to_try = produce_thetas_to_try(multipliers_to_try, D_matrix, D, y)

        # Produce statistics for this run
        trt_intersections_run, ctl_intersections_run, permissible_trees_run, size_of_r_sets_run = produce_crf_counter(num_trees_to_try, thetas_to_try, D, y, X_cf, T, y_0d, parameters)

        if i == 1:
            trt_intersections = trt_intersections_run
            ctl_intersections = ctl_intersections_run
            permissible_trees = permissible_trees_run
            size_of_r_sets = size_of_r_sets_run
        else:
            trt_intersections = (i - 1) / i * trt_intersections + 1 / i * trt_intersections_run
            ctl_intersections = (i - 1) / i * ctl_intersections + 1 / i * ctl_intersections_run
            permissible_trees = (i - 1) / i * permissible_trees + 1 / i * permissible_trees_run
            size_of_r_sets =  (i - 1) / i * size_of_r_sets + 1 / i * size_of_r_sets_run

    return trt_intersections, ctl_intersections, permissible_trees, size_of_r_sets, parameters

        
trt_intersections, ctl_intersections, permissible_trees, size_of_r_sets, parameters = run_sims(num_trees_to_try, multipliers_to_try, 100)



Running Iteration 1
Running RAggregate with theta = 1.9538519297075907 and reg = 0.1
Length of the RPS is 9


100%|██████████████████████████████████████| 500/500 [00:00<00:00, 26958.45it/s]


Running RAggregate with theta = 2.1314748324082804 and reg = 0.1
Length of the RPS is 49


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 5571.72it/s]


Running RAggregate with theta = 2.309097735108971 and reg = 0.1
Length of the RPS is 155


...

100%|███████████████████████████████████████| 500/500 [00:00<00:00, 5759.16it/s]


Running RAggregate with theta = 2.4071001056387358 and reg = 0.1
Length of the RPS is 105


100%|███████████████████████████████████████| 500/500 [00:00<00:00, 2800.75it/s]


Running RAggregate with theta = 2.777423198813926 and reg = 0.1
Length of the RPS is 1014


100%|████████████████████████████████████████| 500/500 [00:01<00:00, 281.12it/s]


In [22]:
print("Average Intersection with Treatment:", trt_intersections)
print("Average Intersection with Control:", ctl_intersections)
print("Average Number of Permissible Trees:", permissible_trees)
print("Average Size of Rashomon Set:", size_of_r_sets)
print("Order of Parameters Considered:", parameters)

Average Intersection with Treatment: [[ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.52  1.15  4.24]
 [ 1.29  2.67 10.32]]
Average Intersection with Control: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Average Number of Permissible Trees: [[ 1.29  2.67 10.32]
 [ 1.29  2.67 10.32]
 [ 1.29  2.67 10.32]
 [ 1.29  2.67 10.32]]
Average Size of Rashomon Set: [[  7.46   7.46   7.46]
 [ 46.6   46.6   46.6 ]
 [126.54 126.54 126.54]
 [823.81 823.81 823.81]]
Order of Parameters Considered: [(1.1, 20), (1.1, 100), (1.1, 500), (1.2, 20), (1.2, 100), (1.2, 500), (1.3, 20), (1.3, 100), (1.3, 500), (1.5, 20), (1.5, 100), (1.5, 500), (1.9538519297075907, 20), (1.9538519297075907, 100), (1.9538519297075907, 500), (2.1314748324082804, 20), (2.1314748324082804, 100), (2.1314748324082804, 500), (2.309097735108971, 20), (2.309097735108971, 100), (2.309097735108971, 500), (2.664343540510351, 20), (2.664343540510351, 100), (2.664343540510351, 500), (1.9615380383456924, 20), (1.9615380383456924, 100), (1.96

## LASSO

This section uses LASSO in order to get an idea of a reasonable threshold for pruning the RPS.

In [101]:
lasso_reg = 1e-1


lasso = linear_model.Lasso(lasso_reg, fit_intercept=False)
lasso.fit(D_matrix, y)
# lasso.fit(X_trt_subset, y_trt_subset)
alpha_est = lasso.coef_
y_tva = lasso.predict(D_matrix)

sqrd_err = mean_squared_error(y, y_tva)
print(sqrd_err)

lasso_loss = sqrd_err + 1e-4 * np.linalg.norm(alpha_est, ord=1)
print(lasso_loss)

# y_tva = lasso.predict(D_matrix_trt_subset)
# # y_tva = lasso.predict(X_trt_subset)

# tva_results = metrics.compute_all_metrics(
#                     y_trt_subset, y_tva,
#     D_trt_subset, true_best, all_policies, profile_map, min_dosage_best_policy, true_best_effect)

2.2421820501307677
2.2428753174251383
